In [1]:
import pandas as pd
import numpy as np 
import torch 
import matplotlib.pyplot as plt 
import collections
import nltk 
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
import string
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk import ngrams
import copy
from collections import defaultdict
from functools import cache
import cProfile
import pstats
import time
from tabulate import tabulate

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/christophhalberstadt/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/christophhalberstadt/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


nltk stopwords

Duck DB for View of Projctions

In [2]:
data = pd.read_json('/Users/christophhalberstadt/Documents/TU Berlin/LSDIPro/tables.json', 
                    lines=True, nrows=10000, )

Real Datatables might be Row and Column Based!!! 

In [3]:
class Config: 

    batch_size = 3000
    stop_words = None 
    stemmer = PorterStemmer()

    create_ngrams = False 
    ngram_size = 2
    min_word_len = 0 

    read_path = '/Users/christophhalberstadt/Documents/TU Berlin/LSDIPro/tables.json'
    

In [ ]:
class Tokenizer: 

    def __init__(self, stemmer): 
        self.stemmer = stemmer 

    def __call__(self, value:str, create_ngrams:bool = False, ngram_size:int = 2) -> list: 
        """
        Hier sollte ihr DocString stehen 
        """
        ######Min_word_Len müsste hier noch enforced werden!!! 
        words = self.process_words(value)
        stemmed_words = [self.stem_cached(word) for word in words]

        if create_ngrams: 
            created_ngrams = ngrams(stemmed_words, ngram_size)
            stemmed_words.extend(list(created_ngrams))
            
        return set(stemmed_words)             ####War vorher ne Liste

    @cache
    def stem_cached(self, word:str):
        """
        Returns the stemmed version of the input word. 

        In: 
            Word: str
        
        Out: 
            The stemmed version of the word: str
        """
        return self.stemmer.stem(word)
    
    @cache
    def process_words(self, text:str)->list: 
        """
        Processes the input text into a lowercase, punctuation free and tokenized list of strings. 

        In: 
            text: str

        Out: 
            words: list(str)
        """
        sentence = text.lower().translate(str.maketrans("", "", string.punctuation))            ####Konfigurierbar in der Init machen 
        words = word_tokenize(sentence)
        return words 

Die Beiden müssen noch in eine Klasse gepackt werden!!!!

In [30]:
batch_size = Config.batch_size 
with open(Config.read_path, "r") as infile:
    for i, line in enumerate(infile):
        if i >= batch_size:
            break

In [31]:
def iter_json(path, csize):
    iter_chunk = pd.read_json(path, lines=True, chunksize=csize)
    for chunk in iter_chunk:
        yield chunk

Bis hier!!!

In [5]:
def create_table_list(dfs):
    """
    Creates a list of tables. 
    Indexing Rank: Table_ID -> Row_ID -> Col_ID 

    In: 
        dfs:pd.dataframe? 
    Out: 
        Table_List: list
    """
    
    return [[[cell.get('text', '') for cell in row] for row in table_data] for table_data in dfs['tableData']]

In [6]:
def create_projections(table_list,
                       #start_table_index, Warum? Ist der von mir? 
                       tokenizer, 
                       #projections,
                       #stemmer, 
                       stop_words: list = None, 
                       min_word_len = 0, 
                       create_ngrams:bool = False, 
                       ngrams_size:int = 2
                       ) -> dict: 
    


    ######Trie Indexing for Fuzzy Matching fehlt 

    projections = dict()

    #ps = stemmer
    
    if stop_words: 
        stemmed_stopwords = {tokenizer.stem_cached(w) for w in stop_words}
    else:
        stemmed_stopwords = set()

    for table_index, table in enumerate(table_list): 
        for row_index, row in enumerate(table): 
            for column_index, cell in enumerate(row): 

                words = tokenizer.process_words(cell)

                if create_ngrams: 
                    ngram_stem_set = set()

                for word in words: 
                    stemmed_word = tokenizer.stem_cached(word)
                        
                    if stemmed_word not in stemmed_stopwords and len(stemmed_word) >= min_word_len: 

                        projections.setdefault(stemmed_word, set()).add((table_index, row_index, column_index))

                        if create_ngrams: 
                            ngram_stem_set.update(stemmed_word)
                
                if create_ngrams: 
                    created_ngrams = ngrams(ngram_stem_set, ngrams_size)
                    for created_ngram in created_ngrams: 
                        projections.setdefault(created_ngram, set()).add((table_index, row_index, column_index))
    
    return projections

Also: 
In: E_Set

Relevant_Tables = dict()

for E in E_Set: 
    X, Y = E 
    for X get all (Table_ID, Row_ID, Col_ID)
    for Y get all (Table_ID, Row_ID, Col_ID)


    for all ID_Tup in X: 
        for all ID_Tup2 in Y: 
            if Table_ID_X == Table_ID_Y and Row_ID_X == Row_ID_Y and Col_ID_X != Col_ID_Y: 
                relevant_Tables.setdefault(Table_ID): (Row_ID, (Col_ID_X, Col_ID_Y))


Probleme hiermit: 
    X und Y können Subkeys sein. 
    Dabei kann X über zwei splaten definiert sein und: 
        X_i in X kann durch Tokenizer zu einer Liste werden!! 

        Also mehr info needed!!!! 



Zusammengesetzte Keys: 
Weenn X oder Y ein zusammengesetzer Key, also 
    if isinstance(X, tuple) oder if isinstance(Y, tuple): 
        for x_i/ y_i in X/Y get all (Table_ID, Row_ID, Col_ID)

Es gilt immer noch!!! Forall x_i in X and y_i in Y. 
To be relevant, forall x_i in X, y_i in Y, Table_ID: Same, Row_ID: Same, Col_ID:Different!!! 

-> Kein Problem, selbe Struktur wie oben, aber es müssen mehr Vergleiche Stattfinden. 
-> Early Opt out to kill Table? 
-> Sorting musst start from big to small, so first compare Table_ID (fast)
-> Then compare Row_ID 
->Then compare Col_ID
if something is wrong, continue with next 


Zusammengesetzte Values: 
z.B. Fc Bayern München. -> Fc, Bayern, München 
Alle sollten im selben Column sein. 
Wenn nicht, dann müsste man die Teil Columns Mergen. 
Wie macht man so eine Struktur? 
Was ist mit der Informationsqualität. Eg. Fc Bayern München mit min_word_size und zusammengesetzt eindeutig -> Fussballverein. 
                                            Bayern München mit min_word_size = 3 und zusammengsetzt -> Liste aller Länder und deren Landeshauptstädte in DE?? 
                                            Liste mit min_word_size = 0 und NICHT zusammengesetzt uneindeutig -> Fc Köln != Fc Bayern München, aber Fc = Fc, also Hit 
                                            Daher ngrams? Wahrscheinlich schon. Fc Bayern eher eindeutig, wenn auch nicht perfekt. Alle ngrams bis 3, also Fc, Bayern, München, Fc Bayern, Bayern München, Fc Bayern München hittet viel (auch unnötiges) aber definitiv auch das richtige. Würde das nicht Fc, Bayern, München auch? Bringen ngrams hier was? hmmmm 
                    
Wenn drei Einzelne, dann wird Table_ID, Col_ID, Row_ID mindestens drei mal gehittet. 
Da aber set(), egal!!! 
Sagen wir, mir ist egal wie mein E aussieht. Ich tokenize alles in listen und hänge die Liste zusammen. Dann müssen einfach, wenn ich n X und m Y habe nach dem Tokenizen mindestens len(Liste) >= n+m sein und damit Table zählt müssen mindestens n+m Hits in Zeile sein. 
Für Tabelle dann Tau Zeilen Hits! 




Also, da mir die einzelnen Wörter total egal sind, ist das Token selbst info, die ich verlieren kann. 
Was ich jetzt brauche ist nur Info: 
Anzahl Subkeys X, Anzahl Subkeys Y, Tokenized Subkeys liste/set? hmmm. set()!! Wenn nach Tokenize die Anzahl kleiner ist, dann muss ich entweder das als minimal Requirement nutzen oder das Example Error Raisen!!! 
(Table_ID, Row_ID, Col_ID) pro Val in Tokenized_Example_Set 
Welche Datenstruktur wäre am schnellsten für den Vergleich? 
Ah, nächstes Problem. 
Da ja l ID_Tuples pro Val in Tokenized_Example_List auftauchen können, muss auch getracked werden, zu welchem die gehören. 




Neue Datenstruktur: 

Example_Dict: Dict()
for Example in Example_Dict: 

Table_ID, Row_ID, Col_ID




Besser: list(Dict_1; ..., Dict_(n+m))
Dict_Structure: (Table_ID, Row_ID): Col_ID

Compare_Keys = Set(Dict_1) & Set(Dict_2) & ... Set(Dict_(n+m))
If Compare_Keys: 
    for key in Compare_Keys: 




Die Anzahl der Col muss nur n+m sein, um einen Example_Hit zu verzeichnen.
Mir können duplikate in zwei columns total egal sein, jedes Dict enthält ein Set. 
Ich muss alle gleichen Keys finden und nur alle sets joinen. Wenn die kardinalität des Sets n+m entspricht ist es ein Hit, sonst ein Miss 

Die einzige Frage: Was returne ich? Da ich das ja anpassen kann wie ich will ist mir das erstmal egal. 

In [7]:
def indexing(tokenized_values:list, projections:dict)->dict:
    """
    Return a dict of all the Examples found in the projections
    In: 
        Cleaned_Values: A List of all the Stemped Versions of one Example given
        Projections: A Dict of Projections of all given Tables
    Out: 
        Index_Dict: A dict of all the positions where the Example was found
                    Form: Key: (Table_ID, Row_ID) -> Value: (Col_ID)
    """ 

    index_dict = dict() 
    
    for value in tokenized_values: 
        value_index = projections.get(value, None)

        if value_index: 
            for table_id, row_id, col_id in value_index: 
                #table_id, row_id, col_id = index_pair 
                index_dict.setdefault((table_id, row_id), set()).add((col_id))
    
    return index_dict

In [42]:
def find_direct_tables(Examples:set,  
                       projections:dict, 
                       Tau: int, 
                       tokenizer:Tokenizer):

    table_dict = dict() 
    E = len(Examples)
    K = None


    if Tau > E: 
        raise ValueError(f"At least Tau: {Tau} examples must be given!")
    

    for Example in Examples: 

        if not K: 
            K = len(Example)                
        else: 
            if K != len(Example): 
                raise ValueError(f"All Examples must be of the same Size!")
        

        

        idx_list = list()               #####Speichert die dict_of_idx pro example_key          ###Hier dict für x und y? 
        for example_key in Example:                 #####Vorher hier in X und Y spalten? 
            tokens_of_example_key = tokenizer(example_key)

            dict_of_idx = indexing(tokens_of_example_key, projections)
            idx_list.append(dict_of_idx)


        key_sets = [set(d.keys()) for d in idx_list]
        unique_shared_keys = set.intersection(*key_sets)
        
        if not unique_shared_keys: 
            continue 


        for unique_shared_key in unique_shared_keys: 
            col_set = set()
            valid = True
            for dict_of_idx in idx_list: 
                cols = dict_of_idx.get(unique_shared_key, None)
                if not cols: 
                    valid = False 
                    break
                else: 
                    col_set = col_set.union(cols)
            
            if valid == True and len(col_set) >= K: ######zweiter Teil dieser Bedingung muss aufgeweicht werden!!! 
                Table_ID, Row_ID = unique_shared_key
                table_dict.setdefault((Table_ID, *list(col_set)), set()).add(Row_ID)


    """
    relevant_tables = set()

    for key in table_dict: 
        elems = table_dict.get(key)
        if len(elems) >= Tau: 
            relevant_tables.add(key)
    """

    relevant_tables = dict()

    for key in table_dict: 
        elems = table_dict.get(key)
        if len(elems) >= Tau: 
            relevant_tables.setdefault(key[0], key[1:])
    
    return relevant_tables

In [ ]:
def find_direct_tables(Examples:dict,  
                       projections:dict, 
                       Tau: int, 
                       tokenizer:Tokenizer):

    table_dict = dict() 
    E = len(Examples)
    K = None


    if Tau > E: 
        raise ValueError(f"At least Tau: {Tau} examples must be given!")
    

    for Example in Examples: 

        #####Nehmen wir an, Example wäre ein dict: x:{key1, ..., keyn}, y:{key1, ..., keym}

        if not K: 
            K = len(Example)                
        else: 
            if K != len(Example): 
                raise ValueError(f"All Examples must be of the same Size!")
        

        

        idx_list = dict               #####Speichert die dict_of_idx pro example_key          ###Hier dict für x und y? 
        for side, keys_per_side in Example:                 #####Vorher hier in X und Y spalten? 
            for example_key in keys_per_side: 

                tokens_of_example_key = tokenizer(example_key)

                len_of_tokens = len(tokens_of_example_key)
                #####A:
                #####Hier können zwei verschiedene Ergeinisse auftreten: 
                #####1: |tokens_of_example_key| = 1
                #####2: |tokens_of_example_key| > 1
                #####Können leere Tokens auftreten? 
                #####Da min_word_len nicht im tokenizer enforced wird, können hier auch fehlerhafte Tokens sein, die niemals in dict auftauchen werden
                #####zB of oder fc sollten bei min_word_len=3 oder of in Stopwords gekilled werden. Key of wäre auch dumm, aber Tubsoken possible 
                #####min_word_len im Tokenizer enforcen? 

                dict_of_idx = indexing(tokens_of_example_key, projections)

                ####B:
                ####Hier Können drei verschiedene Ereignisse auftreten: 
                ####1: dict_of_idx ist leer -> Break Example 
                ####2: für ein (Table_ID, Row_ID) ist len() = 1
                ####3: für ein (Table_ID, Row_ID) ist len() > 1 

                ####Was ist, wenn dict_of_idx leer ist? Break example? 
                ####Wenn A = 1 und B = 1 -> Eindeutig
                ####Wenn A = 1 und B > 1 -> Uneindeutig, Problem!! Aber, kann durch andere Keys und Examples gefiltert werden
                ####Wenn A > 1 und B = 1 -> Treffer, aber sind alle tokens in der einer Spalte? Ist es ein Problem, wenn dem nicht so ist? 
                ####Wenn A > 1 und B > 1 -> Key ist auf mehrere Spalten verteilt? Sind alle Tokens ein Hit? Problem, wenn dem nicht so ist? 


                ####Brauche ich einen Faktor Precission zusätzlich zu Tau? Sortiert sich das selber durch Tau und weitere Examples aus? 

                idx_list.setdefault(side, list()).append(dict_of_idx)       ####Also für jeden Subkey einer Seite alle seine Beispiele 


        key_sets = [set(d.keys()) for d in idx_list]
        unique_shared_keys = set.intersection(*key_sets)
        
        if not unique_shared_keys: 
            continue 


        for unique_shared_key in unique_shared_keys: 
            col_set = set()
            valid = True
            for dict_of_idx in idx_list: 
                cols = dict_of_idx.get(unique_shared_key, None)
                if not cols: 
                    valid = False 
                    break
                else: 
                    col_set = col_set.union(cols)
            
            if valid == True and len(col_set) >= K: #######Ist hier der zweite Vergleich wirklich war? schließlich könnten ja auch zwei Keys in einer Col sein
                                                    #######Was ist die mindest Menge an Hits? Reicht Condition 1, dass jeder Key einen Hit 
                                                    #######Verzeichnet haben muss?
                                                    #######Multi Hits eines Keys mit len(Tokens) > 1 als ein Tuple? 
                                                    #######Müsste für unverschateltem Readout wieder aufgespalten Werden. 
                                                    #######Eher Struktur dict Key: x Value: dict Key: side_key Value: cols???
                Table_ID, Row_ID = unique_shared_key
                table_dict.setdefault((Table_ID, *list(col_set)), set()).add(Row_ID)


    
    relevant_tables = set()

    for key in table_dict: 
        elems = table_dict.get(key)
        if len(elems) >= Tau: 
            relevant_tables.add(key)
    
    return relevant_tables


####Output Information sollte die Anzahl an Example Hits Tau beinhalten. Also Dict in Dict Structure, mit den Keys: Cols, Hits, Rows? 

####Also, welche Informationen brauche ich für die Evaluation? 
####Die Anzahl an Hits pro Example einer Tabelle. -> Table based -> Hits/ Examples 
####
####Welche Information brauche ich für einen Example hit? 
####Alle/ die meisten Keys müssen einen oder mehrere Hits verzeichnen /Mehrere Hits müssen zu einem werden. 
####Brauche ich die Information welches Token wo getroffen hat, um eine Trennung des Keys im allgemeinen zu bestimmen? 
####Also für die nachfolgdenden Querries!
####
####Wenn zwei substantiell unterschiedliche Keys einer Seite in einer Spalte einen Hit landen, dann müssen beide Keys einen Hit verzeichnen
####Aka, Hits = K (Errechnung von K muss geändert werden)
####Wenn |Tokens| > 1 für einen Key, wie viele müssen Hitten um einen Hit für das Example zu bedeuten? 
####zB |Tokens| = 2, und zwei Spalten, fine. Beide Hits in einer Saplte, Fine, Nur ein Hit in einer Spalte? -> Luca fragen
####zB |Tokens| = 1, zwei Spalten. Nehme ich beide auf und verzeichne nur einen Hit, in der Hoffunung, dass weitere Examples das Problem lösen? 
####
####Alternative, der bisherige Ansatz, selbst ohne x und y Trennung und ich löse das Problem einfach in einem zweiten, kleineren, refinement Schritt
####Dort ordne ich jedem Key die Spalte zu und gebe so die Lösung für eine bestimmte Tabelle an?? 
####Logik auf kleiner endlicher Anzahl an Tabellen mit kleiner endlicher Anzahl an Examples ist nicht besonders hart. 
####Aber!!! Bound für Tabelle ist deutlich leichter!!!


Nur ein Eintrag für alle Subtokens, egal wie viele es sind

In [10]:
table_list = create_table_list(data)
print(len(table_list))

10000


In [11]:
my_tokenizer = Tokenizer(Config.stemmer)

In [12]:
projections = create_projections(table_list, my_tokenizer)

In [39]:
indexing_example = [('1929', 'Robert Crawford'), ('1938', 'John Patrick')]
tau = 2

In [43]:
relevant_tables = find_direct_tables(indexing_example, projections, tau, my_tokenizer)

In [44]:
print(relevant_tables)

{0: (1, 2)}


In [ ]:
col1 = [] 
col2 = []
table_id, x_col_id, y_col_id = next(iter(relevant_tables))          ######Lieber Col Store für die Speicherung der Tabelle? 
                                                                    ######Ist "richtiger"/schneller
for row in table_list[table_id]: 
    col1.append(row[x_col_id])
    col2.append(row[y_col_id])

print(col1)
print(col2)

['1929', '1933', '1938', '1945', '1949', '1953', '1958', '1962', '1965', '1969', '1973']
['Robert Crawford', 'Robert Crawford', 'John Patrick', 'John Patrick', 'Robert Nichol Wilson', 'Robert Simpson', 'Robert Simpson', 'Robert Simpson', 'Robert Simpson', 'Robert Simpson', 'Constituency abolished']


Lieber einfach alle Cols auswählen, die identifiziert wurden und auf diesen das Mapping suchen? 